In [1]:
from transformers import GPT2Model, GPT2Tokenizer
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm

if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('Device name:', device)

Device name: cpu


In [2]:
df = pd.read_csv('/root/projects/CAP5619-sp24/P2/codes/amazon_reviews.csv')
del df[df.columns[0]] # first column is just index so we drop

In [3]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, data_list):
        self.data_list = data_list

    def __len__(self):
        return len(self.data_list)

    def __getitem__(self, idx):
        return self.data_list[idx]

text = df['reviewText'].tolist()
text = [item for item in text if isinstance(item, str)] # removing any reviews that are not strings
dataset = MyDataset(text)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])
train_text = [train_dataset[i] for i in range(len(train_dataset))]
test_text = [test_dataset[i] for i in range(len(test_dataset))]

In [4]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2Model.from_pretrained('gpt2')

tokenizer.pad_token = tokenizer.eos_token  # Use the end of sequence token as padding token.
encoded_inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt")


embeddings = []
# obtain embeddings

chunk = 10
for i in tqdm(range(0,len(encoded_inputs['input_ids']),chunk)):
    with torch.no_grad():
        outputs = model(encoded_inputs['input_ids'][i:i+chunk])
        hidden_states = outputs.last_hidden_state
        embeddings.append(torch.mean(hidden_states, dim=1))  # mean pooling

# print(embeddings)
embeddings_cat = torch.cat(embeddings, dim=0)
torch.save(embeddings_cat, 'embeddings.pt')

KeyboardInterrupt: 

In [5]:
embeddings = torch.load("embeddings.pt", map_location=device)

In [29]:
from torch import nn

class GPT2ForClassification(nn.Module):
    def __init__(self, num_labels):
        super(GPT2ForClassification, self).__init__()
        self.gpt2 = GPT2Model.from_pretrained('gpt2')
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(self.gpt2.config.n_embd, num_labels)

    def forward(self, inputs_embeds, attention_mask=None):
        outputs = self.gpt2(inputs_embeds=inputs_embeds, attention_mask=attention_mask)
        hidden_state = outputs.last_hidden_state
        hidden_state = self.dropout(hidden_state)
        logits = self.classifier(hidden_state)
        return logits


In [23]:
# need to remove the same index from the ratings that we removed earlier from the reviews that was "not-a-string"
non_string_review = df['reviewText'].apply(lambda x: not isinstance(x, str))
non_string_indices = non_string_review[non_string_review].index
print("Indices of non-string values in column 'reviewText':", non_string_indices.tolist())

ratings = df['overall'].drop(df['overall'].index[non_string_indices.values[0]]) # removal of problem index from the ratings
print(len(ratings))

Indices of non-string values in column 'reviewText': [125]
4914


In [24]:
from sklearn.model_selection import train_test_split

# Convert tensors to numpy arrays for splitting
embeddings_np = embeddings.numpy()
ratings = ratings.to_numpy().astype(int) # convert to integer numpy array for classification

# Split the data
embeddings_train, embeddings_test, labels_train, labels_test = train_test_split(
    embeddings_np, ratings, test_size=0.20)

# Convert numpy arrays back to tensors
embeddings_train = torch.tensor(embeddings_train)
embeddings_test = torch.tensor(embeddings_test)
labels_train = torch.tensor(labels_train)-1
labels_test = torch.tensor(labels_test)-1

In [25]:
#labels_train = labels_train - 1
labels_train

tensor([4, 3, 4,  ..., 4, 4, 4])

In [55]:
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader
import numpy as np

# Create dataset and dataloader for more efficient training
train_dataset = TensorDataset(embeddings_train, labels_train)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

# Initialize the model and optimizer
model = GPT2ForClassification(num_labels=5)
optimizer = Adam(model.parameters(), lr=1e-5)

#find the frequency of each class
unique, counts = np.unique(ratings, return_counts=True)
# reverse frequency of each class
total = sum(counts)
freq = [total/c for c in counts]
freq

loss_fn = nn.CrossEntropyLoss(weight=torch.tensor(freq, dtype=torch.float32))
epoch_losses = []
batch_losses = []

# Training loop
model.train()
for epoch in range(10):  # number of epochs
    batch_losses = []
    for embeddings_batch, labels_batch in train_loader:
        optimizer.zero_grad()
        logits = model(embeddings_batch)
        loss = loss_fn(logits, labels_batch)
        batch_losses.append(loss.item())
        loss.backward()
        optimizer.step()
    epoch_losses.append(np.mean(batch_losses))
    print(f"Epoch {epoch + 1}, Loss: {epoch_losses[-1]}")



Epoch 1, Loss: 2.0469691525629865
Epoch 2, Loss: 1.667957430205694
Epoch 3, Loss: 1.661915639067084
Epoch 4, Loss: 1.6366413434346516
Epoch 5, Loss: 1.6317570073817804
Epoch 6, Loss: 1.6349371569912607
Epoch 7, Loss: 1.6155807337140649
Epoch 8, Loss: 1.6058927299530525
Epoch 9, Loss: 1.620350834799976
Epoch 10, Loss: 1.6118604061080188


In [56]:
from sklearn.metrics import accuracy_score

# Switch to evaluation mode
model.eval()
with torch.no_grad():
    logits_test = model(embeddings_test)
    predictions = torch.argmax(logits_test, dim=1)
    accuracy = accuracy_score(labels_test, predictions.numpy())
    print(f"Test Accuracy: {accuracy}")


Test Accuracy: 0.7965412004069176


In [57]:
predictions.unique()

tensor([3, 4])